In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
import networkx as nx

from scipy import stats

import pybel as pb
import json
import time
import csv

import torch
import pyro

from bel2scm import graph_node as cn
from bel2scm import gen_test_data as td
from bel2scm import causal_graph as cg

pyro.set_rng_seed(101)
torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
tot_data = td.data_gen(10000)

# Unit Tests

In [ ]:
# Unit Test in icream graph

str_list = ['temp =| cloudy','cloudy => rainy','temp => icream','rainy =| icream']
type_dict = {}
type_dict['temp'] = 'continuous'
type_dict['cloudy'] = 'continuous'
type_dict['rainy'] = 'continuous'
type_dict['icream'] = 'continuous'
graph_test = cg.str_graph(str_list=str_list,graph_type='SCM',type_dict=type_dict)
graph_test.add_confound([['icream','cloudy']])

print(graph_test.d_sep(['icream'],['cloudy'],['rainy','temp'],[]))
#Ans: False

print(graph_test.id_alg(['icream'],['cloudy']))
#Ans: P_{cloudy}(icream) = sum_{rainy} P(rainy|cloudy) sum_{cloudy}′P(icream|cloudy',rainy)P(cloudy')


print(graph_test.idc_alg(['icream'],['cloudy'],['temp']))
#Ans: P_{cloudy}(icream|temp) = [sum_{rainy} P(rainy|cloudy,temp) sum_{cloudy'} P(icream|cloudy',rainy,temp)P(cloudy'|temp)]  
# /[sum_{rainy,icream} P(rainy|cloudy,temp) sum_{cloudy'}P(icream|cloudy',rainy,temp)P(cloudy'|temp)]

In [ ]:
# Unit Test in XYZ Graph

str_list = ['X =| W','D => Z','W => Y','Z =| Y','V =| Z']
type_dict = {}
type_dict['X'] = 'continuous'
type_dict['W'] = 'continuous'
type_dict['D'] = 'continuous'
type_dict['Z'] = 'continuous'
type_dict['Y'] = 'continuous'
type_dict['V'] = 'continuous'

graph_test = cg.str_graph(str_list=str_list,graph_type='SCM',type_dict=type_dict)
graph_test.add_confound([['X','Y']])

print(graph_test.d_sep(['D'], ['Z'],['Y']))
# Ans: True

print(graph_test.id_alg(['Y'],['X']))
# Ans: P_X(Y) = sum_W P(W|X) sum_X' P(Y|X',W)P(X')

print(graph_test.idc_alg(['Y'],['X'],['W']))
# Ans: P_X(Y|W) = [sum_{D,V,Z} P(Z|D,V) P(V) P(D) sum_X' P(Y|D,V,W,X′,Z) P(X'|D,V,Z)]
#/[sum_{D,V,Z,Y} P(Z|D,V) P(V) P(D) sum_x' P(Y|D,V,W,X′,Z) P(X'|D,V,Z)]

In [ ]:
# Unit Test in XYZ Graph from Shpitser thesis and paper

str_list = ['X =| W','D => Z','W => Y','Z =| Y']
type_dict = {}
type_dict['X'] = 'continuous'
type_dict['W'] = 'continuous'
type_dict['D'] = 'continuous'
type_dict['Z'] = 'continuous'
type_dict['Y'] = 'continuous'

graph_test = str_graph(str_list=str_list,graph_type='SCM',type_dict=type_dict)
graph_test.add_confound([['X','Y']])


do_temp = [['Y',['X*','Z*','D']],['Z',['D']]]
obs_temp = ['X','D']

do_temp2 = [['Y',['X*','Z*','D']]]
obs_temp2 = ['X']

do_gam = [['Y',['X*']]]
do_del = [['Z',['D']]]
obs_del = ['X','D']


g_temp,c_temp = graph_test.make_pw_graph(do_temp)
pos = nx.kamada_kawai_layout(g_temp)    
dict_temp = {}
for item in g_temp.nodes:
    dict_temp[item] = item
    
plt.figure()
nx.draw(g_temp,pos,labels=dict_temp,width=4,font_size=8)
nx.draw(c_temp,pos,labels=dict_temp,font_size=8)
plt.show()

g_temp,c_temp,gamma_temp = graph_test.make_cf_graph(do_temp,obs_temp)
pos = nx.circular_layout(g_temp)    
dict_temp = {}
for item in g_temp.nodes:
    dict_temp[item] = item
    
plt.figure()
nx.draw(g_temp,pos,labels=dict_temp,width=4,font_size=8)
nx.draw(c_temp,pos,labels=dict_temp,font_size=8)
plt.show()

print(graph_test.id_star_alg(do_temp2,obs_temp2))
# Ans: sum_{W}P_{Z*,W*}(Y,X)P_{X*}(W)

print(graph_test.idc_star_alg(do_gam,do_del,[],obs_del))
# Ans: [sum_{W}P_{Z,D,W*}(Y,X)P_{X*}(W)]/[sum_{Y}[sum_{W}P_{Z,D,W*}(Y,X)P_{X*}(W)]]

Parallel Worlds Graph: <img src="shpitser_pw_graph.png">
Counterfactual Graph<img src="shpitser_cf_graph.png">